In [178]:
from bs4 import BeautifulSoup
import requests
from typing import List, Dict, Tuple
import urllib.parse as urlparse
import re
from random import randint
import json
from datetime import date

In [153]:
# index_page_url = 'https://data.bopp-obec.info/emis/index.php'
# response = requests.get(index_page_url)

# with open('index.html', 'w') as f:
#   f.write(response.content.decode('utf-8'))

In [9]:
# with open('index.html', 'r') as f:
#   html_page = f.read()

In [13]:
soup = BeautifulSoup(html_page, 'html')

In [41]:
def select_for_name(options: List) -> str:
  for option in options:
    if 'selected' in option.attrs.keys():
      return option.get_text()
  return ''

In [109]:
def params_in_url(url) -> List[Tuple]:
  params = url[(url.find('?')+1):]
  if not params: return []
  params = params.split('&')
  params = [tuple(param.split('=', maxsplit=1)) for param in params]
  return params

In [ ]:
re.sub('', '', )

In [148]:
interested_select_tag: str = 'จังหวัด/ศธจ.'
interested_url_param: str = 'province'
all_interested_id: List[Dict] = list()

for select_tag_soup in soup.find_all('select'):
  options = select_tag_soup.findAll('option')
  select_tag_for = select_for_name(options=options)
  if select_tag_for != interested_select_tag:
    continue

  for option in options:
    if 'value' not in option.attrs.keys(): continue
    if not option.attrs['value']: continue
    key_name = re.sub('\s*\d+\s*\.\s*', '', option.text)
    for key, val in params_in_url(option.attrs['value']):
      if key == interested_url_param:
        all_interested_id.append({
          'name': key_name,
          'id': val
          })


In [155]:
school_data_url_format = \
  'https://data.bopp-obec.info/emis/school_edu_p.php?province={}'

In [247]:
# school_list_index = {}
# for item in all_interested_id:
#   url = school_data_url_format.format(item['id'])
#   response = requests.get(url)
#   if response.status_code == 200:
#     file_path = 'school_list/' + item['id'] + '.html'
#     with open(file_path, 'w') as html_file:
#       html_file.write(f'<!-- url: {url} -->\n')
#       html_file.write(f'<!-- scraped: {date.today().isoformat()} -->\n')
#       html_file.write(response.content.decode('utf-8'))
#     school_list_index[item['id']] = {
#       'name': item['name'],
#       'file_path': file_path,
#       'updated_at': date.today().isoformat(),
#     }
#   else: print('status_code not 200', item)

# with open('school_list_index.json', 'w') as index_file:
#   json.dump(school_list_index, index_file, ensure_ascii=False, indent=2)

# with open('school_list_index.json', 'r') as index_file:
#   school_list_index = json.load(index_file)

In [248]:
# class SchoolDataIndex:
#   def __init__(self,):
#     self.dta = {}
#     self.load()
#     pass

#   def load(self):
#     with open('school_list_index.json', 'r') as index_file:
#       self.data = json.load(index_file)

#   def save(self):
#     with open('school_list_index.json', 'w') as index_file:
#       json.dump(self.data, index_file, ensure_ascii=False, indent=2)


In [250]:
# sdi = SchoolDataIndex()

In [257]:
def parse_table(table_soup):
  thead = table_soup.find('thead')
  assert len(thead) > 0
  headers = [head.text for head in thead.find_all('th')]
  table_data = []
  for row in table_soup.find('tbody').find_all('tr'):
    row_tds = row.find_all('td')
    assert len(row_tds) == len(headers)
    row_data = {}
    for col, td in zip(headers, row.find_all('td')):
      row_data[col.strip()] = td.text.strip()
      a_tag = td.find('a')
      if a_tag is not None:
        row_data['href'] = a_tag.attrs['href']
    table_data.append(row_data)
  return table_data

In [258]:
def province_school_list(file_path):
  with open(file_path, 'r') as file:
    soup = BeautifulSoup(file, 'html')

  for table in soup.find_all('table'):
    if table.find('table'): continue
    thead = table.find('thead')
    if thead is not None:
      break

  table_data = parse_table(table)
  return table_data


In [259]:
for id in sdi.data:
  sdi.data[id]['schools'] = province_school_list(sdi.data[id]['file_path'])

In [260]:
sdi.save()